In [ ]:
%pip install openai langchain

In [ ]:
# LangChain Settings

from langchain.llms import AzureOpenAI

llm = AzureOpenAI(temperature=0.3, deployment_name="textdavinci003")

In [ ]:
llm.predict(
    "What would be a good company name for a company that makes colorful socks?"
)